In [2]:
################ Library imports ################
import numpy as np
import math as math

from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

In [10]:
# ----------------------------------------------------------------------
#  AP locations
L_ap = np.array([[0, 0], [0, 2], [2, 0], [2, 2]])

# ----------------------------------------------------------------------
#  VRP locations
L_vrp = np.array([[1, 0], [0, 1], [1, 1], [1, 2], [2, 1]])

# ----------------------------------------------------------------------
#  VRP locations
d_RTT = np.array([math.sqrt(8), 2, 2, 0])

In [11]:
"""
Gaussian Process Kernel Definition

Kernel parameters:
    - L --> Length Scale
    - sigma_n --> Training Noise Variance
"""

l = 0.1
sigma_f = 2

kernel = ConstantKernel(constant_value=sigma_f,constant_value_bounds=(1e-3, 1e3)) \
            * RBF(length_scale=l, length_scale_bounds=(1e-3, 1e3))  

In [68]:
# Make the prediction on the meshed x-axis (ask for MSE as well)
y_pred, sigma = gp.predict(L_vrp, return_std=True)